In [ ]:
import os

import numpy as np
import pandas as pd
import pyreadr

import matplotlib.pyplot as plt
from scipy import stats

from gglasso.helper.utils import sparsity, zero_replacement, normalize, log_transform
from gglasso.problem import glasso_problem

### Import preprocessed soil data

In [ ]:
metadata = pd.read_table('../data/soil/88soils_modified_metadata.txt', index_col=0)
metadata.head(2)

In [ ]:
soil = pd.read_csv('../data/soil/soil_id_116.csv', sep=',', index_col = 0).T
soil.head()

In [ ]:
ph = metadata["ph"]
ph = ph.reindex(soil.columns)
ph.head()

In [ ]:
#check if any ids are missing
assert not ph.isnull().values.any()

# assert that samples of ph and soil are identical
assert set(soil.columns) == set(ph.index)

### CLR-transformation of X

##### Dataframe `soil` need to be of shape (p,N) for normalizing to simplex + clr transform

In [ ]:
soil.shape

In [ ]:
X = normalize(soil)
X.shape

In [ ]:
X = log_transform(X)
(p,N) = X.shape
(p,N)

### Calculate covariance and scale to correlations

In [ ]:
S0 = np.cov(X.values, bias = True)
# scale covariances to correlations
scale = np.tile(np.sqrt(np.diag(S0)),(S0.shape[0],1))
scale = scale.T * scale

S = S0 / scale
S.shape

### GGLasso low-rank with lambda = 0.14447343, mu =  2.36, rank=6

In [ ]:
P = glasso_problem(S, N, latent = True, do_scaling = False)

print(P)

lambda1_range = [0.14447343]
mu1_range = [2.36]


modelselect_params = {'lambda1_range': lambda1_range, 'mu1_range': mu1_range}

P.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = 0.25)

# regularization parameters are set to the best ones found during model selection
print(P.reg_params)

In [ ]:
# gg_lowrank = pd.read_csv("~/Downloads/gglasso_lr.csv", sep=',', header=None)
gg_lowrank = P.solution.lowrank_
gg_lowrank.shape

In [ ]:
gg_rank = np.linalg.matrix_rank(gg_lowrank)
print('Rank of low-rank component: {0}'.format(gg_rank))

### SE low-rank with lambda = 0.14447343, rank=6

In [ ]:
SE_lowrank = pyreadr.read_r('SE_lowrank.rds')
SE_lowrank = SE_lowrank[None]
SE_lowrank = np.array(SE_lowrank)
SE_lowrank.shape

In [ ]:
se_rank = np.linalg.matrix_rank(SE_lowrank)
print('Rank of low-rank component: {0}'.format(se_rank))

### Compare two solutions

In [ ]:
np.allclose(SE_lowrank, gg_lowrank, atol=1e-01)

In [ ]:
import seaborn as sns
fig, axs = plt.subplots(1,2, figsize = (20,5))
sns.heatmap(SE_lowrank, ax = axs[0], cmap = "coolwarm", vmin = -0.1, vmax = 0.1)
sns.heatmap(gg_lowrank, ax = axs[1], cmap = "coolwarm", vmin = -0.1, vmax = 0.1)

### Robust PCA in [SE](https://github.com/zdk123/SpiecEasi/blob/ff528b23fafbd455efcca9dd356bef28951edf82/R/SparseLowRankICov.R)

In [ ]:
# robustPCA <- function(X, L, inverse=TRUE) {
#   Lsvd <- svd(L)
#   ind <- Lsvd$d>1e-9
#   if (inverse) {
#     loadings <- diag(sqrt(1/Lsvd$d[ind])) %*% t(Lsvd$v[,ind])
#   } else {
#     loadings <- diag(sqrt(Lsvd$d[ind])) %*% t(Lsvd$v[,ind])
#   }

#   scores <- X %*% t(loadings)
#   return(list(scores=scores, loadings=loadings))
# }

### Robust PCA in GGLasso

In [ ]:
def robust_PCA(X, L, inverse=True):
    #SVD of low-rank component
    v_tilde, sigma, v_tilde_T  = np.linalg.svd(L, full_matrices=True)
    
    ind = np.argwhere(sigma > 1e-9)
    
    if inverse:
        loadings = np.diag(np.sqrt(1/sigma[ind]).reshape(len(ind),)) @ v_tilde[:, :len(ind)].T
    else:
        loadings = np.diag(np.sqrt(sigma[ind]).reshape(len(ind),)) @ v_tilde[:, :len(ind)].T
        
    zu = X.T @ loadings.T
    
    
    return zu, loadings

In [ ]:
def robust_PCA(X, L, inverse=True):
    sig, V = np.linalg.eigh(L)
    ind = np.argwhere(sig > 1e-9)

    if inverse:
        loadings = V[:,ind] @ np.diag(np.sqrt(1/sig[ind]))
    else:
        loadings = V[:,ind] @ np.diag(np.sqrt(sig[ind]))

    zu = X.values.T @ loadings
    
    return zu, loadings

### Plot SE/pH correlation

In [ ]:
zu_SE, se_loadings = robust_PCA(X, SE_lowrank, inverse=True)
zu_SE.shape

In [ ]:
for i in range(se_rank):
    plt.scatter(zu_SE[:,i], ph)
    plt.xlabel("Learned low-rank PCA component")
    plt.ylabel("pH")
    plt.show()

In [ ]:
for i in range(se_rank):
    print("Spearman correlation between pH and {0}th component: {1}, p-value: {2}".format(i+1, stats.spearmanr(ph, zu_SE[:,i])[0], 
                                                                              stats.spearmanr(ph, zu_SE[:,i])[1]))

### Plot GGLasso/pH correlation

In [ ]:
zu_gg, gg_loadings = robust_PCA(X, gg_lowrank, inverse=True)

In [ ]:
for i in range(gg_rank):
    plt.scatter(zu_gg[:,i], ph)
    plt.xlabel("Learned low-rank PCA component")
    plt.ylabel("pH")
    plt.show()

In [ ]:
for i in range(0, se_rank):
    print("Spearman correlation between pH and {0}th component: {1}, p-value: {2}".format(i+1, stats.spearmanr(ph, zu_gg[:,i])[0], 
                                                                              stats.spearmanr(ph, zu_gg[:,i])[1]))